In [ ]:
import re
import os
import pandas as pd
import json
import uuid

In [ ]:
pwd

'/content'

In [ ]:

data = pd.read_json("./VQA_RAD Dataset Public.json")

In [ ]:
print(data.shape)

(2248, 14)


In [ ]:
# divide images into group
grouped_data = data.groupby("image_name")


formatted_data = []


for image_name, group in grouped_data:
    conversations = []

    # add `image_organ` as the context
    image_organ = group.iloc[0]["image_organ"]
    if pd.notna(image_organ) and image_organ != "NULL":
        conversations.append({
            "from": "human",
            "value": "<image>\nWhat organ is shown in the image?"
        })
        conversations.append({
            "from": "gpt",
            "value": image_organ
        })

    # build `conversation`
    for _, row in group.iterrows():
        # add question
        conversations.append({
            "from": "human",
            "value": f"<image>\n{row['question']}"
        })
        # add answer
        conversations.append({
            "from": "gpt",
            "value": row["answer"]
        })

    # add into the target list
    formatted_data.append({
        "id": str(uuid.uuid4()),
        "image": {image_name},
        "conversation": conversations
    })


with open("formatted_vqa_rad_with_organ.json", "w", encoding="utf-8") as f:
    json.dump(formatted_data, f, ensure_ascii=False, indent=4)

print("Data saved successfully！")

In [ ]:
# Import necessary libraries
import os
import pandas as pd
import json
import shutil

In [ ]:
# Load the parquet dataset
parquet_file = "train.parquet"  # Replace with your actual parquet file path
data = pd.read_parquet(parquet_file)


In [ ]:
# Create the output folder for images
image_folder = "image"
os.makedirs(image_folder, exist_ok=True)

# Initialize the output JSON list
output_json = []

In [ ]:
for idx, row in data.iterrows():
    image_data = row["image"]["bytes"]
    image_filename = f"image_{idx}.jpg"
    image_output_path = os.path.join(image_folder, image_filename)

    with open(image_output_path, "wb") as img_file:
        img_file.write(image_data)

    output_json.append({
        "id": f"entry_{idx}",
        "image": image_filename,
        "conversations": [
            {"from": "human", "value": f"<image>\n{row['question']}"},
            {"from": "gpt", "value": row["answer"]}
        ]
    })

output_json_file = "out.json"
with open(output_json_file, "w", encoding="utf-8") as f:
    json.dump(output_json, f, indent=4, ensure_ascii=False)



In [ ]:
import json

input_json_file = "out.json"
output_json_file = "output_corrected.json"

with open(input_json_file, "r", encoding="utf-8") as f:
    data = json.load(f)

for entry in data:
    for conversation in entry["conversations"]:
        if conversation["from"] == "human":
            # if human's value without "<image>\n"
            if not conversation["value"].startswith("<image>\n"):
                conversation["value"] = f"<image>\n{conversation['value']}"

with open(output_json_file, "w", encoding="utf-8") as f:
    json.dump(data, f, indent=4, ensure_ascii=False)

print(f"fix the format {output_json_file}")


In [ ]:
from google.colab import files
import shutil

folder_to_zip = "image"
output_zip = "image_folder.zip"

shutil.make_archive(output_zip.replace(".zip", ""), 'zip', folder_to_zip)

files.download(output_zip)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

###split the data into train and test dataset


####unzip the image data

In [ ]:
!pwd

/content


In [ ]:
!unzip VQA_RAD\ Image\ Folder.zip -d /content/image/


Archive:  VQA_RAD Image Folder.zip
  inflating: /content/image/synpic676.jpg  
  inflating: /content/image/synpic15006.jpg  
  inflating: /content/image/synpic9872.jpg  
  inflating: /content/image/synpic12210.jpg  
  inflating: /content/image/synpic13385.jpg  
  inflating: /content/image/synpic16170.jpg  
  inflating: /content/image/synpic16221.jpg  
  inflating: /content/image/synpic16174.jpg  
  inflating: /content/image/synpic16520.jpg  
  inflating: /content/image/synpic16407.jpg  
  inflating: /content/image/synpic16810.jpg  
  inflating: /content/image/synpic17153.jpg  
  inflating: /content/image/synpic17145.jpg  
  inflating: /content/image/synpic17664.jpg  
  inflating: /content/image/synpic17693.jpg  
  inflating: /content/image/synpic17675.jpg  
  inflating: /content/image/synpic18319.jpg  
  inflating: /content/image/synpic18461.jpg  
  inflating: /content/image/synpic18250.jpg  
  inflating: /content/image/synpic17848.jpg  
  inflating: /content/image/synpic17738.jpg  
  

In [ ]:
import json
import os
import random
import shutil

with open("formatted_vqa_rad_with_organ.json", "r", encoding="utf-8") as f:
    data = json.load(f)

image_folder = "/content/image"
train_folder = "/content/train_folder"
test_folder = "/content/test_folder"

os.makedirs(train_folder, exist_ok=True)
os.makedirs(test_folder, exist_ok=True)

train_ratio = 0.8
test_ratio = 1 - train_ratio

random.shuffle(data)

num_train = int(len(data) * train_ratio)
train_data = data[:num_train]
test_data = data[num_train:]

with open("train_conversations.json", "w", encoding="utf-8") as f:
    json.dump(train_data, f, ensure_ascii=False, indent=4)

with open("test_conversations.json", "w", encoding="utf-8") as f:
    json.dump(test_data, f, ensure_ascii=False, indent=4)

# Move the image into to folder
for dataset, folder in [(train_data, train_folder), (test_data, test_folder)]:
    for item in dataset:
        image_name = item["image"]
        image_path = os.path.join(image_folder, image_name)
        if os.path.exists(image_path):
            shutil.copy(image_path, folder)
        else:
            print(f"Warning: Image {image_name} not found in {image_folder}")

print("Split the dataset successfully！")


####download the dataset

In [ ]:
from google.colab import files
import shutil


folder_to_zip = "train_folder"
output_zip = "train_image_folder.zip"


shutil.make_archive(output_zip.replace(".zip", ""), 'zip', folder_to_zip)


files.download(output_zip)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>